In [1]:
import folium
import requests
import json

In [2]:
# let's start by creating an empty map using the IMO basemapbas

# basemap tile url and attribution
basemap_tiles = "https://geo.vedur.is/geoserver/www/imo_basemap_epsg3857/{z}/{x}/{y}.png"
basemap_attribution = "Icelandic Met Office | National Land Survey of Iceland | © OpenStreetMap contributors"

#create map and assign options
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)

#initiate and display map. You can also save as html with map.save("index.html")
map

In [3]:
# let's add a WMS layers and zoom in our map a bit
# we will use on of several glacier outlines available
# to get a full list of avaialble layers goto https://geo.vedur.is/geoserver/wms?request=GetCapabilities&service=WMS

map = folium.Map(location=[66.15, -22.2], zoom_start=11, tiles=basemap_tiles, attr=basemap_attribution)

# define parameters for a WMS layer
wms_overlay = folium.raster_layers.WmsTileLayer(
               url="https://geo.vedur.is/geoserver/wms", # url to wms server
               name="Glacier outlines around 1890", # name of wms layer for layer controls
               fmt='image/png',  # image/png is the default for format
               layers="glaciology:glacieroutline_lia", # name of wms layer
               transparent=True, # allow transparancy
               #opacity=0.5, # set transparancy
               overlay=True, # overlay wms layer on top of basemap
               )

wms_overlay.add_to(map)

map

In [24]:
# now let's try WFS

# constructing a request
base_url = 'https://geo.vedur.is/geoserver/ows?'
params = {
    'service':'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typeName': 'meteorology:IcelandicMetOffice_CAP_areas',
    #'cql_filter': 'region_ord LIKE \'2%\'',
    'cql_filter': 'region_ord = 5',
    'outputFormat': 'application/json'
}

# send request
r = requests.get(base_url, params)
# lets take a look at the geoJSON
r.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'IcelandicMetOffice_CAP_areas.9',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-23.39, 65.6],
       [-23.65, 65.59],
       [-24.01, 65.71],
       [-24.11, 65.81],
       [-23.86, 66.06],
       [-23.27, 66.3],
       [-23.1, 66.56],
       [-22.87, 66.52],
       [-22.58, 66.47],
       [-22, 65.99],
       [-22, 65.7],
       [-22.54, 65.69],
       [-23.01, 65.65],
       [-23.39, 65.6]]]]},
   'geometry_name': 'the_geom',
   'properties': {'name_en': 'Westfjords',
    'name_is': 'Vestfirðir',
    'region_ord': 5}}],
 'totalFeatures': 1,
 'numberMatched': 1,
 'numberReturned': 1,
 'timeStamp': '2023-02-06T17:11:21.304Z',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}}}

In [25]:
# save response as json
data = r.json()
with open('forecast_areas.json', 'w') as f:
    json.dump(data, f)
f.close()

In [26]:
# make a new map and add the geoJSON from the previous WFS request
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)
folium.GeoJson(data, name="Forecast areas").add_to(map)
folium.LayerControl().add_to(map)
map

In [10]:
base_url = 'https://geo.vedur.is/geoserver/ows?'
params = {
    'service':'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typeName': 'tos:v_stations_wgs84',
    #'maxFeatures': '20',
    'cql_filter': 'subtype LIKE \'SIL%\' AND date_end IS NULL',
    'outputFormat': 'application/json'
}
r = requests.get(base_url, params)
r.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'v_stations_wgs84.fid-5431a075_1861c8fa2ea_-519a',
   'geometry': {'type': 'Point', 'coordinates': [-21.7511, 64.3018]},
   'geometry_name': 'geom_wgs84',
   'properties': {'gid': 16074,
    'id_entity': 16074,
    'name': 'Morastaðir',
    'station_identifier': 'mor',
    'marker': 'mor',
    'lat': 64.30177,
    'lon': -21.7511,
    'altitude': 130,
    'domain': 'geophysical',
    'subtype': 'SIL stöð',
    'operational_class': 'A',
    'owner': None,
    'date_start': '2017-11-22T14:00:00Z',
    'date_end': None,
    'service_bulletin': None}},
  {'type': 'Feature',
   'id': 'v_stations_wgs84.fid-5431a075_1861c8fa2ea_-5199',
   'geometry': {'type': 'Point', 'coordinates': [-22.0595, 63.92]},
   'geometry_name': 'geom_wgs84',
   'properties': {'gid': 18389,
    'id_entity': 18389,
    'name': 'Móhálsadalur',
    'station_identifier': 'moh',
    'marker': 'moh',
    'lat': 63.91995,
    'lon': -22.05954,
    'al

In [11]:
data = r.json() # add data to get nested list
with open('SIL_stations.json', 'w') as f:
    json.dump(data, f)
f.close()

In [9]:
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)
folium.GeoJson(data, name="Seismic stations").add_to(map)
folium.LayerControl().add_to(map)
map